In [10]:
# Model definition
import os
import numpy as np
import pandas as pd

# SKLEARN
from sklearn.preprocessing  import ( StandardScaler, )
from sklearn.model_selection import ( train_test_split,KFold,cross_val_score, )

#KERAS 
from keras import backend
from keras import models
from keras import layers
from keras import (Input,)
from keras.applications import (VGG16,)
from keras.callbacks import (ModelCheckpoint,EarlyStopping,LearningRateScheduler,ReduceLROnPlateau,CSVLogger,)
from keras.datasets import (boston_housing,mnist, imdb,)
from keras.layers import (Dense,Flatten,Embedding, SimpleRNN, LSTM, GRU, MaxPooling1D, Conv1D, GlobalMaxPooling1D )
from keras.models import (load_model,Model,Sequential, )
from keras.optimizers import (RMSprop,)
from keras.preprocessing.image import (ImageDataGenerator,image,)
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasRegressor


import matplotlib.pyplot as plt
%matplotlib inline

#from utils import plot_history

In [2]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 50

# Multi input models

In [5]:
def build_model():
    text_input = Input(shape=(None,), dtype='int32', name='text')
    embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
    encoded_text = layers.LSTM(32)(embedded_text)                            
    
    question_input = Input(shape=(None,),dtype='int32',name='question')
    embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
    encoded_question = layers.LSTM(16)(embedded_question)

    concatenated = layers.concatenate([encoded_text, encoded_question],axis=-1)
    answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated)
    model = Model([text_input, question_input], answer)                      
    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
    
    print(model.summary())
    return model

In [6]:
build_model()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 10000)  640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 10000)  320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LS

# Multi ouput models

In [12]:
vocabulary_size = 50000
num_income_groups = 10

def build_model2():
    posts_input = Input(shape=(None,), dtype='int32', name='posts')
    embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
    x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(256, 5, activation='relu')(x)
    x = layers.Conv1D(256, 5, activation='relu')(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(256, 5, activation='relu')(x)
    x = layers.Conv1D(256, 5, activation='relu')(x)
    x = layers.GlobalMaxPooling1D()(x)
    x = layers.Dense(128, activation='relu')(x)
    age_prediction = layers.Dense(1, name='age')(x) 
    income_prediction = layers.Dense(num_income_groups, activation='softmax',name='income')(x)
    gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

    model = Model(posts_input, [age_prediction, income_prediction, gender_prediction])
    model.compile(optimizer='rmsprop',loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
    model.compile(optimizer='rmsprop',loss={'age': 'mse','income': 'categorical_crossentropy','gender': 'binary_crossentropy'})         
    print(model.summary())
    return model
model = build_model2()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 50000)  12800000    posts[0][0]                      
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, None, 128)    32000128    embedding_6[0][0]                
__________________________________________________________________________________________________
max_pooling1d_3 (MaxPooling1D)  (None, None, 128)    0           conv1d_6[0][0]                   
__________________________________________________________________________________________________
conv1d_7 (

# Adding callbacks

In [11]:
callbacks = [
    EarlyStopping(monitor='acc', patience=1),
    ModelCheckpoint(filepath='mymodel.h5', monitor='val_loss', save_best_only=True),
    ReduceLROnPlateau( monitor='val_loss',factor=0.1, patience=10),
]
model.fit(x,y,callbacks = callbacks)

NameError: name 'model' is not defined